##### O intuito desse programa é criar playlists baseados em seus gostos de artistas e músicas usando a lib spotipy

existem dois "modos de uso". os dois usam a função recommendTracks.

o parâmetro listTracks deve ser no formato 
[ ['passionfruit', 'drake'],['aquela fé','don l'] ] 
(ou seja, cada elemento da lista é uma lista com nome da musica e artista(opcional) )

e listArtists no formato ['frank ocean', 'nill'].

os nomes de artistas e musicas devem ser escritos corretamente, mas são insensíveis a letras maiúsculas ou minúsculas

OBS: tamanho das duas listas somados pode ser até 5

Modos de uso:

(1): atribuir a chamada da recommendTracks a uma variável, e preenchendo somente os parâmetros listMusics e listArtists. Isso vai imprimir sua playlist na tela e salvar nessa variável. Caso vc goste do que leu, basta usar a função recommentationsToPlaylist( suaVariavel , nomeDaPlaylist )

(2): tentar a sorte e criar uma playlist sem olhar ela antes. para isso, chamar a função recommendTracks como acima, mas adicionar o parâmetro name e colocar o parâmetro createPlaylist = True  

##### Login

Para logar em seu spotify, deve-se rodar a célula de login, que irá te redirecionar para tela de login. Ao aceitar que o app acesse sua conta, você será redirecionado para uma url (talvez seu navegador diga que deu erro ao carregar. Mas não deu. vc só precisa da url). Você deve copiar a url para qual foi redirecionado e colocar como input

________________________________________________________

Configuração e importações

In [1]:
# caso não tenha o spotipy instalado
# pip install spotipy --upgrade

In [2]:
import spotipy

Célula de login abaixo

In [3]:
sp = spotipy.Spotify(
    auth_manager=spotipy.oauth2.SpotifyOAuth(
        client_id='3a92f783c17f4b95bc75d536dc51be3d',
        client_secret='89e6b1f4d2914d158a34c71f71190d2f',
        redirect_uri="https://localhost:8000/callback",
        scope="user-library-read playlist-modify-public user-library-modify playlist-modify-private"
        )
    )
    
sp.me()
# vai te redirecionar para uma tela de login do spotify. 
# copia o link depois de logar e cola aqui como input

{'display_name': 'lucasdonaire',
 'external_urls': {'spotify': 'https://open.spotify.com/user/lordbugadaao'},
 'followers': {'href': None, 'total': 27},
 'href': 'https://api.spotify.com/v1/users/lordbugadaao',
 'id': 'lordbugadaao',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab6775700000ee85dfd54fe1cd1df45dd2b42bcf',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:lordbugadaao'}

____________________________________________________

In [4]:
def searchArtist(name):
  # buscar um artista por nome -> nome indiferente a letras maiúsculas e minúsculas, mas tem que estar escrito certo, inclusive acentos
  res = sp.search(name,limit=10)
  for i in range(10): # testa os 10 primeiros artistas
    artist = res['tracks']['items'][i]['album']['artists'][0]
    if name.lower() == artist['name'].lower():
      return artist['id']
  print('ERRO: artista achado:'+res['tracks']['items'][0]['album']['artists'][0]['name'])
  return -1

In [5]:
def searchTrack(name,artist=''):
  # buscar uma música por nome -> nome indiferente a letras maiúsculas e minúsculas, mas tem que estar escrito certo, inclusive acentos
  stringSearch = name+' '+artist+' &type=track'
  rec = sp.search(stringSearch,limit=10)
  for i in range(10):
    track = rec['tracks']['items'][i]
    trackBool = track['name'].lower() == name.lower() 
    artistBool = track['artists'][0]['name'].lower() == artist.lower()
    if (trackBool and artistBool) or (artist=='' and trackBool):
      return track['id']
  track = rec['tracks']['items'][0]
  print('ERRO: track achada:'+track['name'])
  print('ERRO: artista achado:'+track['artists'][0]['name'])
  return -1


In [6]:
def auxTrackSearch(item: list):
  # função auxiliar para usar com map()
  if len(item) == 1:
    item.append('')
  return searchTrack(item[0],item[1])

In [7]:
def recommentationsToPlaylist(rec,name,description=''):
  # pega os resultados de um retorno da função sp.recommendations() e salva numa playlist
  playlist = sp.user_playlist_create(sp.me()['id'],name=name,description=description)
  listTracks = []
  for item in rec['tracks']:
    listTracks.append(item['id'])
  sp.playlist_add_items(playlist['id'],listTracks)
  print('=================================== \n playlist ',name,' criada com sucesso.')

In [8]:
def recommendTracks(listMusics=[], listArtists=[], createPlaylist=False, name='', playlist_size=20):
  # list tracks no formato [['passionfruit', 'drake'],['aquela fé','don l']]
  # list artists no formato ['frank ocean', 'nill']
  # tamanho das duas listas somados pode ser até 5
  if listMusics == [] and listArtists ==[]:
    print('listas vazias')
    return
  if len(listMusics) + len(listArtists) > 5:
    print('maximo de musicas+artistas é 5')
    return
  listIdsMusics = list(map(auxTrackSearch,listMusics))
  listIdsArtists = list(map(searchArtist,listArtists))
  if -1 in listIdsMusics: 
    print('deu ruim na track '+str(listMusics[listIdsMusics.index(-1)]))
    print(listIdsMusics)
    return
  if -1 in listIdsArtists: 
    print('deu ruim no artista '+listArtists[listIdsArtists.index(-1)])
    print(listIdsArtists)
    return
  rec = sp.recommendations(seed_artists=listIdsArtists, seed_tracks=listIdsMusics, limit=playlist_size)
  print('_________ PLAYLIST __________')
  for item in rec['tracks']:
    print(item['name'],' --- ',item['artists'][0]['name'])
  print('____________________________')
  if createPlaylist:
    if name == '':
      name = 'playlist baseada nas musicas '+str(listMusics)[1:-1]+'e nos artistas'+str(listArtists)[1:-1]
    description = description='playlist baseada nas musicas '+str(listMusics)[1:-1]+'e nos artistas'+str(listArtists)[1:-1]
    recommentationsToPlaylist(rec,name,description)
  return rec

### Testes

Modo de uso ( 1 )

In [9]:
rec = recommendTracks(
    listMusics=[['Passionfruit', 'Drake'],['aquela fé','don l']],
    listArtists=['Nill','Don L','Makalister']
)

_________ PLAYLIST __________
Levar a Sério  ---  FBC
Eu Tive um Sonho  ---  Rodrigo Ogi
Subirusdoistiozin  ---  Criolo
Fôlego  ---  SKiT
Instalações Noturnas, Assuntos de Rua  ---  Makalister
See You Again (feat. Kali Uchis)  ---  Tyler, The Creator
Take Ten  ---  Black Alien
Eu Vou  ---  Hot
Luzes  ---  BK
#tamotransandodefato  ---  Lívia Cruz
Só  ---  Kamau
Chicago Freestyle (feat. Giveon)  ---  Drake
De Fato  ---  Raffa Moreira
ball w/o you  ---  21 Savage
Reminder  ---  The Weeknd
Rap É Compromisso  ---  Sabotage
Só de Garra  ---  Makalister
Todo Mundo Ama o Sol  ---  Tarcis
Virou Canção  ---  Rodrigo Ogi
Long Time - Intro  ---  Playboi Carti
____________________________


In [10]:
# rodar isso vai criar uma playlist com os resultados acima chamada playloka
recommentationsToPlaylist(rec,'playloka')

 playlist  playloka  criada com sucesso.


Modo de uso ( 2 )

In [11]:
# rodar isso vai criar uma playlist com os resultados que vão ser impressos chamada playLOKA
rec = recommendTracks(
    listMusics = [['Passionfruit', 'Drake'],['aquela fé','don l']],
    listArtists = ['Nill','Don L','Makalister'],
    createPlaylist = True,
    name = 'playLOKA'
)

_________ PLAYLIST __________
MIA  ---  Bad Bunny
Autoestima em Dó  ---  Froid
Call Out My Name  ---  The Weeknd
Mandinga  ---  Murica
Primavera  ---  PrimeiraMente
Long Time - Intro  ---  Playboi Carti
Tropiqal  ---  Davzera
Morra Bem, Viva Rápido  ---  Don L
Tiro na Cara  ---  ADL
Pedagoginga  ---  Thiago Elniño
Espírito Vândalo  ---  Funkero
Odisséia  ---  Rap Box
Auto  ---  Chinv
Esú  ---  Baco Exu do Blues
Jovem Frozen - Versão Extendida  ---  Makalister
Fazia Sentido  ---  Don L
Atari  ---  Nill
That's What I Like  ---  Bruno Mars
Autumn Ring Mini (Sozinho no Touge)  ---  Yung Buda
Mask Off  ---  Future
____________________________
 playlist  playLOKA  criada com sucesso.
